# Descrição do dataset

O dataset é constituto pelo livro:  

- O adversário secreto (Agatha Christie)

Todas as obras utilizadas estão em dominio público e foram extraídos do Project Gutenberg, disponível em: https://www.gutenberg.org/. 

In [ ]:
import spacy as sp
import en_core_web_sm
import numpy as np
import random
from tensorflow import keras
from tensorflow.keras import callbacks
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential 
from keras.layers import Dense, LSTM, Embedding
from pickle import dump, load

In [ ]:
#Função de leitura de arquivo de texto
def read_file(filepath):
    with open (filepath) as f:
        str_text = f.read()
        return str_text

In [ ]:
#Função de limpeza do texto
#Tira do texto pontuação, novas linhas e caracteres especiais
def separate_punc(doc_text):
    return[token.text.lower() 
           for token in nlp(doc_text) 
           if token.text not in '\n\n \n\n\n!"-#$%&()--.*+---,-/:;<=>?@[\\]^_`{|}~\t\n ' ]

In [ ]:
#Próximo passo torkenizar e limpar o texto
nlp = sp.load('en_core_web_sm', disable=['parser', 'tagger', 'ner','lemmatizer'])

In [ ]:
Secret = read_file('/The Secret Adversary.txt')
Secret

'PROLOGUE\nIT was 2 p.m. on the afternoon of May 7, 1915. The Lusitania had\nbeen struck by two torpedoes in succession and was sinking\nrapidly, while the boats were being launched with all possible\nspeed. The women and children were being lined up awaiting their\nturn. Some still clung desperately to husbands and fathers;\nothers clutched their children closely to their breasts. One girl\nstood alone, slightly apart from the rest. She was quite young,\nnot more than eighteen. She did not seem afraid, and her grave,\nsteadfast eyes looked straight ahead.\n"I beg your pardon."\nA man\'s voice beside her made her start and turn. She had\nnoticed the speaker more than once amongst the first-class\npassengers. There had been a hint of mystery about him which had\nappealed to her imagination. He spoke to no one. If anyone spoke\nto him he was quick to rebuff the overture. Also he had a nervous\nway of looking over his shoulder with a swift, suspicious glance.\nShe noticed now that he was 

In [ ]:
#nlp.max_length = 1403894

In [ ]:
tokens = separate_punc(Secret)
tokens

['prologue',
 'it',
 'was',
 '2',
 'p.m.',
 'on',
 'the',
 'afternoon',
 'of',
 'may',
 '7',
 '1915',
 'the',
 'lusitania',
 'had',
 'been',
 'struck',
 'by',
 'two',
 'torpedoes',
 'in',
 'succession',
 'and',
 'was',
 'sinking',
 'rapidly',
 'while',
 'the',
 'boats',
 'were',
 'being',
 'launched',
 'with',
 'all',
 'possible',
 'speed',
 'the',
 'women',
 'and',
 'children',
 'were',
 'being',
 'lined',
 'up',
 'awaiting',
 'their',
 'turn',
 'some',
 'still',
 'clung',
 'desperately',
 'to',
 'husbands',
 'and',
 'fathers',
 'others',
 'clutched',
 'their',
 'children',
 'closely',
 'to',
 'their',
 'breasts',
 'one',
 'girl',
 'stood',
 'alone',
 'slightly',
 'apart',
 'from',
 'the',
 'rest',
 'she',
 'was',
 'quite',
 'young',
 'not',
 'more',
 'than',
 'eighteen',
 'she',
 'did',
 'not',
 'seem',
 'afraid',
 'and',
 'her',
 'grave',
 'steadfast',
 'eyes',
 'looked',
 'straight',
 'ahead',
 'i',
 'beg',
 'your',
 'pardon',
 'a',
 'man',
 "'s",
 'voice',
 'beside',
 'her',
 'mad

In [ ]:
len(tokens)

78441

In [ ]:
#Predição da proxima palavra
train_len = 25+1 

text_sequences = []

for i in range(train_len, len(tokens)):
    
   
    seq = tokens[i-train_len:i]
    
    
    text_sequences.append(seq)

In [ ]:
text_sequences[80]

['she',
 'did',
 'not',
 'seem',
 'afraid',
 'and',
 'her',
 'grave',
 'steadfast',
 'eyes',
 'looked',
 'straight',
 'ahead',
 'i',
 'beg',
 'your',
 'pardon',
 'a',
 'man',
 "'s",
 'voice',
 'beside',
 'her',
 'made',
 'her',
 'start']

In [ ]:
text_sequences[81]

['did',
 'not',
 'seem',
 'afraid',
 'and',
 'her',
 'grave',
 'steadfast',
 'eyes',
 'looked',
 'straight',
 'ahead',
 'i',
 'beg',
 'your',
 'pardon',
 'a',
 'man',
 "'s",
 'voice',
 'beside',
 'her',
 'made',
 'her',
 'start',
 'and']

In [ ]:
text_sequences[82]

['not',
 'seem',
 'afraid',
 'and',
 'her',
 'grave',
 'steadfast',
 'eyes',
 'looked',
 'straight',
 'ahead',
 'i',
 'beg',
 'your',
 'pardon',
 'a',
 'man',
 "'s",
 'voice',
 'beside',
 'her',
 'made',
 'her',
 'start',
 'and',
 'turn']

In [ ]:
#Convertendo os tokens em um sistema númerico
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)

In [ ]:
sequences = tokenizer.texts_to_sequences(text_sequences)

In [ ]:
sequences[0]

[6763,
 8,
 9,
 3503,
 6762,
 24,
 1,
 917,
 5,
 230,
 2461,
 1328,
 1,
 527,
 22,
 60,
 916,
 56,
 101,
 6758,
 12,
 2460,
 6,
 9,
 1893,
 1570]

In [ ]:
tokenizer.index_word

{1: 'the',
 2: 'to',
 3: 'i',
 4: 'a',
 5: 'of',
 6: 'and',
 7: 'you',
 8: 'it',
 9: 'was',
 10: 'he',
 11: 'that',
 12: 'in',
 13: "'s",
 14: 'she',
 15: 'her',
 16: 'his',
 17: 'tuppence',
 18: 'tommy',
 19: 'but',
 20: "n't",
 21: 'with',
 22: 'had',
 23: 'at',
 24: 'on',
 25: 'said',
 26: 'for',
 27: 'be',
 28: 'as',
 29: 'is',
 30: 'do',
 31: 'not',
 32: 'there',
 33: 'him',
 34: 'me',
 35: 'have',
 36: 'they',
 37: 'we',
 38: 'what',
 39: 'julius',
 40: 'up',
 41: 'all',
 42: 'out',
 43: 'no',
 44: 'one',
 45: 'so',
 46: 'would',
 47: 'my',
 48: 'mr.',
 49: 'know',
 50: 'an',
 51: 'sir',
 52: 'if',
 53: 'this',
 54: 'then',
 55: 'did',
 56: 'by',
 57: 'from',
 58: "'",
 59: 'about',
 60: 'been',
 61: 'were',
 62: 'well',
 63: 'james',
 64: 'man',
 65: 'are',
 66: "'ve",
 67: 'them',
 68: 'could',
 69: 'your',
 70: 'or',
 71: 'now',
 72: 'down',
 73: 'think',
 74: 'get',
 75: 'girl',
 76: "'ll",
 77: 'will',
 78: 'just',
 79: 'got',
 80: 'like',
 81: 'vandemeyer',
 82: 'jane',
 83

In [ ]:
for i in sequences[0]:
    print(f'{i} : {tokenizer.index_word[i]}')

6763 : prologue
8 : it
9 : was
3503 : 2
6762 : p.m.
24 : on
1 : the
917 : afternoon
5 : of
230 : may
2461 : 7
1328 : 1915
1 : the
527 : lusitania
22 : had
60 : been
916 : struck
56 : by
101 : two
6758 : torpedoes
12 : in
2460 : succession
6 : and
9 : was
1893 : sinking
1570 : rapidly


In [ ]:
tokenizer.word_counts

OrderedDict([('prologue', 1),
             ('it', 31590),
             ('was', 30603),
             ('2', 30),
             ('p.m.', 5),
             ('on', 12590),
             ('the', 97598),
             ('afternoon', 216),
             ('of', 40350),
             ('may', 1128),
             ('7', 63),
             ('1915', 142),
             ('lusitania', 404),
             ('had', 13301),
             ('been', 5164),
             ('struck', 225),
             ('by', 5584),
             ('two', 3321),
             ('torpedoes', 20),
             ('in', 27373),
             ('succession', 74),
             ('and', 40342),
             ('sinking', 103),
             ('rapidly', 104),
             ('while', 546),
             ('boats', 104),
             ('were', 4914),
             ('being', 1378),
             ('launched', 26),
             ('with', 14482),
             ('all', 8164),
             ('possible', 780),
             ('speed', 104),
             ('women', 104),
         

In [ ]:
#Contando palavras únicas
vocabulary_size = len(tokenizer.word_counts)
vocabulary_size

6764

In [ ]:
seq_matrix = np.array(sequences)
seq_matrix

array([[6763,    8,    9, ...,    9, 1893, 1570],
       [   8,    9, 3503, ..., 1893, 1570,  399],
       [   9, 3503, 6762, ..., 1570,  399,    1],
       ...,
       [  49,   38,    3, ...,    5,  918, 2462],
       [  38,    3,   73, ...,  918, 2462,   56],
       [   3,   73,    8, ..., 2462,   56, 6764]])

In [ ]:
x_features = seq_matrix[:,:-1]
x_features

array([[6763,    8,    9, ...,    6,    9, 1893],
       [   8,    9, 3503, ...,    9, 1893, 1570],
       [   9, 3503, 6762, ..., 1893, 1570,  399],
       ...,
       [  49,   38,    3, ..., 6761,    5,  918],
       [  38,    3,   73, ...,    5,  918, 2462],
       [   3,   73,    8, ...,  918, 2462,   56]])

In [ ]:
y_labels = seq_matrix[:,-1]
y_labels

array([1570,  399,    1, ..., 2462,   56, 6764])

In [ ]:
y_labels = to_categorical(y_labels,num_classes=vocabulary_size+1)
y_labels

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [ ]:
x_features.shape

(78415, 25)

In [ ]:
seq_len = x_features[1]
seq_len

array([   8,    9, 3503, 6762,   24,    1,  917,    5,  230, 2461, 1328,
          1,  527,   22,   60,  916,   56,  101, 6758,   12, 2460,    6,
          9, 1893, 1570])

In [ ]:
seq_len.size

25

In [ ]:
"""def create_model(vocabulary_size, seq_len):
    model = Sequential()
    model.add(Embedding(vocabulary_size, seq_len.size, input_length = seq_len.size))
    model.add(LSTM(150,return_sequences=True))
    model.add(LSTM(150))
    model.add(Dense(150,activation='relu'))
    
    model.add(Dense(vocabulary_size,activation = 'softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    model.summary()
    
    return model"""

"def create_model(vocabulary_size, seq_len):\n    model = Sequential()\n    model.add(Embedding(vocabulary_size, seq_len.size, input_length = seq_len.size))\n    model.add(LSTM(150,return_sequences=True))\n    model.add(LSTM(150))\n    model.add(Dense(150,activation='relu'))\n    \n    model.add(Dense(vocabulary_size,activation = 'softmax'))\n    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])\n    \n    model.summary()\n    \n    return model"

In [ ]:
#model = create_model(vocabulary_size+1, seq_len);

In [ ]:
#model.fit(x_features,y_labels,batch_size=128,epochs=300,verbose=1)

In [ ]:
def create_model_tuned1(vocabulary_size, seq_len):
    model = Sequential()
    model.add(Embedding(vocabulary_size, seq_len.size, input_length = seq_len.size))
    model.add(LSTM(150,return_sequences=True))
    model.add(LSTM(150,return_sequences=True))
    model.add(LSTM(150))
    model.add(Dense(150,activation='relu'))
    model.add(Dense(150,activation='relu'))
    
    opt = keras.optimizers.SGD(learning_rate=0.2, momentum=0.001, nesterov=True, name="SGD")
    
    model.add(Dense(vocabulary_size,activation = 'softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    model.summary()
    
    return model

In [ ]:
model_tuned1 = create_model_tuned1(vocabulary_size+1, seq_len);

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 25, 25)            169125    
                                                                 
 lstm (LSTM)                 (None, 25, 150)           105600    
                                                                 
 lstm_1 (LSTM)               (None, 25, 150)           180600    
                                                                 
 lstm_2 (LSTM)               (None, 150)               180600    
                                                                 
 dense (Dense)               (None, 150)               22650     
                                                                 
 dense_1 (Dense)             (None, 150)               22650     
                                                                 
 dense_2 (Dense)             (None, 6765)              1

In [ ]:
earlystopping = callbacks.EarlyStopping(monitor='accuracy', patience=30, restore_best_weights = True)

In [ ]:
model_tuned1.fit(x_features,y_labels,batch_size=128,epochs=300,verbose=1, callbacks=[earlystopping])

Epoch 1/300
613/613 [==============================] - 9s 14ms/step - loss: 6.4703 - accuracy: 0.0438
Epoch 2/300
613/613 [==============================] - 8s 14ms/step - loss: 6.4614 - accuracy: 0.0446
Epoch 3/300
613/613 [==============================] - 9s 15ms/step - loss: 6.4568 - accuracy: 0.0455
Epoch 4/300
613/613 [==============================] - 8s 13ms/step - loss: 6.4531 - accuracy: 0.0454
Epoch 5/300
613/613 [==============================] - 9s 14ms/step - loss: 6.4511 - accuracy: 0.0459
Epoch 6/300
613/613 [==============================] - 9s 14ms/step - loss: 6.4477 - accuracy: 0.0470
Epoch 7/300
613/613 [==============================] - 8s 14ms/step - loss: 6.4463 - accuracy: 0.0470
Epoch 8/300
613/613 [==============================] - 8s 14ms/step - loss: 6.4440 - accuracy: 0.0467
Epoch 9/300
613/613 [==============================] - 8s 14ms/step - loss: 6.4436 - accuracy: 0.0474
Epoch 10/300
613/613 [==============================] - 8s 14ms/step - loss: 6.441

In [ ]:
model_tuned1.save('capitulo_teste_collab.h5')

In [ ]:
def generate_text(model, tokenizer, seq_len, seed_text, num_gen_words):
    
    output_text = []
    
    input_text = seed_text
    
    for i in range(num_gen_words):
        
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]
        
        pad_encoded = pad_sequences([encoded_text], maxlen=seq_len.size, truncating='pre')
        
        pred_word_ind = model.predict(pad_encoded, verbose=0)[0]
        classes=np.argmax(pred_word_ind)
        
        pred_word = tokenizer.index_word[classes] 
        
        input_text += ' ' + pred_word
        
        output_text.append(pred_word)
        
    return ' '.join(output_text) 

In [ ]:
# gerando o texto seed
random.seed(42)
random_pick = random.randint(0,len(text_sequences))
random_seed_text = text_sequences[random_pick]
random_seed_text

['be',
 'no',
 'second',
 'taxi',
 'therefore',
 'he',
 'would',
 'have',
 'to',
 'run',
 'what',
 'happened',
 'in',
 'actual',
 'fact',
 'to',
 'a',
 'young',
 'man',
 'who',
 'ran',
 'incessantly',
 'and',
 'persistently',
 'through',
 'the']

In [ ]:
seed_text = ' '.join(random_seed_text)
seed_text

'be no second taxi therefore he would have to run what happened in actual fact to a young man who ran incessantly and persistently through the'

In [ ]:
generate_text(model_tuned1,tokenizer,seq_len,seed_text=seed_text,num_gen_words=1000)

"london streets in a main road he might hope to create the illusion that he was merely running for a bus but and weakening and suppressed moan be known ah tuppence well you say she asked to know that anything there was a genuine i guess that think in a moment i make up and that i went into questioning that in the speaker and ' i told me to form the change i heard that does apt more plugged that of that you are sure now and bothering away to me do n't say who start that and a dictaphone are you and do right and for miss the smell of onions you call you just then amount julius directed to put the papers i opened her and opened the underworld and tuppence shook his head if no that to is nothing and dead cross again i soon said sir james something where may seem to get what had are n't she entrusted if how do n't be dreadful answers still o'clock and ha did a sign of america oh what are you proposing to get on maybe but told once tuppence directed sir finn 's more enough that an little sto